# 🧠 NeuroScan - Entrenamiento de Modelo IA

Este cuaderno te guiará paso a paso para entrenar una Inteligencia Artificial capaz de clasificar radiografías (por ejemplo: Normal vs. Neumonía/Tumor) y exportarla para usarla en tu App de Flutter.

### Pasos:
1.  Configuración y carga de datos.
2.  Creación del modelo (Red Neuronal Convolucional).
3.  Entrenamiento.
4.  Evaluación.
5.  **Exportación a TensorFlow Lite (.tflite)**.

## 1. Configuración y Carga de Datos

Primero, necesitamos conectar Google Drive si tus imágenes están ahí, o subir el dataset directamente.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os

# Montar Google Drive (Opcional, si tus datos están ahí)
from google.colab import drive
drive.mount('/content/drive')

# RUTA DE TUS IMÁGENES
# Asegúrate de tener una carpeta con subcarpetas por clase, ejemplo:
# /dataset
#    /Normal
#    /Anomalia
data_dir = '/content/drive/MyDrive/NeuroScan_Dataset' # <--- CAMBIA ESTO POR TU RUTA REAL

In [ ]:
# Configuración de parámetros
img_height = 224
img_width = 224
batch_size = 32

# Cargar datos de entrenamiento
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Cargar datos de validación
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print("Clases encontradas:", class_names)

## 2. Creación del Modelo (CNN)
Usaremos una arquitectura simple pero efectiva para clasificación de imágenes.

In [ ]:
num_classes = len(class_names)

model = models.Sequential([
  # Normalización de pixel (0-255 -> 0-1)
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  
  # Capas Convolucionales (Extraen características)
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  # Capas Densas (Clasificación)
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

## 3. Entrenamiento
Entrenamos el modelo con nuestros datos.

In [ ]:
epochs = 10 # Puedes aumentar esto si necesitas más precisión
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## 4. Exportación a TensorFlow Lite (.tflite)
Este es el paso más importante para tu App. Convertimos el modelo a un formato ligero.

In [ ]:
# Convertir el modelo
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Guardar el archivo .tflite
with open('neuroscan_model.tflite', 'wb') as f:
  f.write(tflite_model)

print("¡Modelo exportado exitosamente! Descarga 'neuroscan_model.tflite' y ponlo en tu carpeta assets de Flutter.")

### 5. Guardar etiquetas (labels.txt)
También necesitas un archivo de texto con los nombres de las clases para que Flutter sepa qué significa cada predicción.

In [ ]:
with open('labels.txt', 'w') as f:
  for class_name in class_names:
    f.write(class_name + '\n')

print("Archivo labels.txt creado.")